In [1]:
import numpy as np
from multiprocessing import shared_memory
import shared_array as sa


In [2]:
a = np.array([1.0], dtype = np.float32)# Start with an existing NumPy array
print(a.nbytes)


4


In [3]:
n = 8
shm = shared_memory.SharedMemory(create=True, size=4*n) # float32 = 4, float64 = 8
b = np.ndarray(n, dtype=np.float32, buffer=shm.buf)
b[0] = 2
print(a)

[1.]


In [4]:
print(shm.name)

psm_fc1710a3


In [5]:
myarr = sa.sharedArray([5,3])

In [6]:
myarr[:,:]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [7]:
myarr[:,:] = 10

In [8]:
print(myarr.handle.name)

psm_1b8a59eb


In [9]:
newarr = sa.sharedArray(myarr.handle)

In [10]:
print(newarr.handle.name)

psm_1b8a59eb


In [11]:
print(newarr[:,:])

[[10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]
 [10. 10. 10.]]


In [12]:
newarr.close()

In [13]:
myarr.close()

In [14]:
myarr.unlink()

In [15]:
#mp.set_start_method('spawn')


# np array over queue

In [16]:
import multiprocessing as mp
import shared_array as sa

def foo(send_q,return_q):
    import shared_array as sa
    s = 0.0
    while True:
        call_arr = send_q.get()
        if call_arr[0] =='array':
            arr = call_arr[1]
            s += np.sum(arr[:])
        else:
            return_q.put(s)
            break

            
send_q = mp.Queue()
return_q = mp.Queue()
p = mp.Process(target=foo, args=(send_q,return_q))
p.start()
import time
t0 = time.time()
for i in range(30):
    myarr = np.empty([5000,3000], float)
    myarr[:,:] = 1
    send_q.put(('array',myarr))
send_q.put(('done'))


print(return_q.get())
p.join()
t1 = time.time()
new_np_time = t1-t0
print(f'time = {t1-t0:.3f} np array over queue')

450000000.0
time = 7.683 np array over queue


# same np array over queue

In [17]:
import multiprocessing as mp
import shared_array as sa

def foo(send_q,return_q):
    import shared_array as sa
    s = 0.0
    while True:
        call_arr = send_q.get()
        if call_arr[0] =='array':
            arr = call_arr[1]
            s += np.sum(arr[:])
        else:
            return_q.put(s)
            break

            
send_q = mp.Queue()
return_q = mp.Queue()
p = mp.Process(target=foo, args=(send_q,return_q))
p.start()
import time
t0 = time.time()
myarr = np.empty([5000,3000], float)
myarr[:,:] = 1
for i in range(30):
    send_q.put(('array',myarr))
send_q.put(('done'))


print(return_q.get())
p.join()
t1 = time.time()
same_np_time = t1-t0
print(f'time = {t1-t0:.3f} same np array over queue')

450000000.0
time = 7.639 same np array over queue


# new shared array handle over queue

In [18]:
import multiprocessing as mp
import shared_array as sa

def foo(send_q,return_q):
    import shared_array as sa
    s = 0.0
    while True:
        call_handle = send_q.get()
        if call_handle[0] =='array':
            handle = call_handle[1]
            arr = sa.sharedArray(handle)
            s += np.sum(arr[:])
            arr.close()
            arr.unlink()
        else:
            return_q.put(s)
            break

            
send_q = mp.Queue()
return_q = mp.Queue()
p = mp.Process(target=foo, args=(send_q,return_q))
p.start()
import time
t0 = time.time()
for i in range(30):
    myarr = sa.sharedArray([5000,3000], float)
    myarr[:,:] = 1
    send_q.put(('array',myarr.handle))
    myarr.close()
send_q.put(('done'))

print(return_q.get())
p.join()
t1 = time.time()
new_shared_time = t1-t0
print(f'time = {t1-t0:.3f} new shared array handle over queue')

450000000.0
time = 1.935 new shared array handle over queue


# same shared array handle over queue

In [19]:
import multiprocessing as mp
import shared_array as sa

def foo(send_q,return_q):
    import shared_array as sa
    s = 0.0
    while True:
        call_handle = send_q.get()
        if call_handle[0] =='array':
            handle = call_handle[1]
            arr = sa.sharedArray(handle)
            s += np.sum(arr[:])
            arr.close()
            #arr.unlink()
        else:
            return_q.put(s)
            break

            
send_q = mp.Queue()
return_q = mp.Queue()
p = mp.Process(target=foo, args=(send_q,return_q))
p.start()
import time
t0 = time.time()
myarr = sa.sharedArray([5000,3000], float)
for i in range(30):
    myarr[:,:] = 1
    send_q.put(('array',myarr.handle))
myarr.close()
send_q.put(('done'))
print(return_q.get())
myarr.unlink()
p.join()
t1 = time.time()
same_shared_time = t1-t0
print(f'time = {t1-t0:.3f} same shared array handle over queue')

450000000.0
time = 0.826 same shared array handle over queue


In [20]:

print(f'time = {new_np_time:.3f} s new np array over queue')
print(f'time = {same_np_time:.3f} s same np array over queue')
print(f'time = {new_shared_time:.3f} s new shared array handle over queue')
print(f'time = {same_shared_time:.3f} s same shared array handle over queue')

time = 7.683 s new np array over queue
time = 7.639 s same np array over queue
time = 1.935 s new shared array handle over queue
time = 0.826 s same shared array handle over queue
